In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import copy, pickle, os, time
import argparse 
from google.colab import drive
import random
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from __future__ import print_function
import torch, os, copy, time, pickle
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
from torchvision.datasets.folder import IMG_EXTENSIONS
from torchvision.datasets import ImageFolder
from sklearn.metrics import confusion_matrix
import glob, pickle
import seaborn as sn
import argparse
start_time= time.time()
import pandas as pd
import numpy as np
import random
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from skimage.io import imread
import PIL
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model_unet = load_model('/content/gdrive/MyDrive/unet_30epochs.hdf5', compile=False)

# **Train covid**

In [ ]:
seed=24
batch_size= 420
n_classes=2

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#Define a function to perform additional preprocessing after datagen.
#For example, scale images, convert masks to categorical, etc. 
def preprocess_data(img):
    img = img / 255. 
    return img


from tensorflow.keras.preprocessing.image import ImageDataGenerator

def trainGenerator(train_img_path):
    
    img_data_gen_args = dict(horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)
    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        class_mode = None,
        color_mode = 'grayscale',
        target_size=(128,128),
        batch_size = batch_size,
        seed = seed)
    
    
    train_generator = image_generator
    
    for img in train_generator:
        img2 = preprocess_data(img)
        yield (img2)

In [ ]:
train_img_path = '/content/gdrive/MyDrive/data2/train/covid'
train_img_gen = trainGenerator(train_img_path)

In [ ]:
print(train_img_gen)

<generator object trainGenerator at 0x7fd56c2b4ad0>


In [ ]:
train_image_batch = train_img_gen.__next__()

Found 420 images belonging to 1 classes.


In [ ]:
print(len(train_image_batch))

420


In [ ]:

images66 = []
for i in range(420):
  test_img = train_image_batch[i]
  test_img_norm=test_img[:,:,0][:,:,None]
  test_img_input=np.expand_dims(test_img_norm, 0)
  print(test_img_input.shape)
  prediction = (model_unet.predict(test_img_input))
  predicted_img=np.argmax(prediction, axis=3)[0,:,:]
  images66.append(predicted_img)

In [ ]:
print(type(images66))

<class 'list'>


In [ ]:
print(len(images66))

420


In [ ]:
import time

i = 0
for item in images66:
  img_w, img_h = 128, 128
  data = np.zeros((img_h, img_w,3), dtype=np.uint8)
  data = item
  img = Image.fromarray((data * 255).astype(np.uint8))
  img.save('/content/gdrive/MyDrive/data3/train/covid/{}.png'.format(i))
  time.sleep(0.2) 
  print(i)
  i = i +1

In [ ]:
len(os.listdir('/content/gdrive/MyDrive/data3/train/covid'))

420

# **Train non covid**

In [ ]:
seed=24
batch_size= 2000
n_classes=2

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#Define a function to perform additional preprocessing after datagen.
#For example, scale images, convert masks to categorical, etc. 
def preprocess_data(img):

    img = img / 255. 

      
    return img


from tensorflow.keras.preprocessing.image import ImageDataGenerator

def trainGenerator(train_img_path):
    
    img_data_gen_args = dict(horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        class_mode = None,
        color_mode = 'grayscale',
        target_size=(128,128),
        batch_size = batch_size,
        seed = seed)

    
    train_generator = image_generator
    
    for img in train_generator:
        img2 = preprocess_data(img)
        yield (img2)

In [ ]:
train_img_path = '/content/gdrive/MyDrive/data2/train/non'
train_img_gen = trainGenerator(train_img_path)

In [ ]:
train_image_batch = train_img_gen.__next__()

Found 2000 images belonging to 1 classes.


In [ ]:

images78 = []
j = 0
for i in range(2000):
  j = j + 1
  test_img = train_image_batch[i]

  test_img_norm=test_img[:,:,0][:,:,None]
  test_img_input=np.expand_dims(test_img_norm, 0)
  print(j)
  prediction = (model_unet.predict(test_img_input))
  predicted_img=np.argmax(prediction, axis=3)[0,:,:]
  images78.append(predicted_img)


In [ ]:
print(len(images78))

2000


In [ ]:
import time

i = 0
for item in images78:
  img_w, img_h = 128, 128
  data = np.zeros((img_h, img_w,3), dtype=np.uint8)
  data = item
  img = Image.fromarray((data * 255).astype(np.uint8))
  img.save('/content/gdrive/MyDrive/data3/train/non/non{}.png'.format(i))
  time.sleep(0.2) 
  print(i)
  i = i +1

# **Test non covid**

In [ ]:
seed=24
batch_size= 3000
n_classes=2

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#Define a function to perform additional preprocessing after datagen.
#For example, scale images, convert masks to categorical, etc. 
def preprocess_data(img):
    #Scale images
    img = img / 255. 

      
    return img

#Define the generator.
#We are not doing any rotation or zoom to make sure mask values are not interpolated.
#It is important to keep pixel values in mask as 0, 1, 2, 3, .....
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def trainGenerator(train_img_path):
    
    img_data_gen_args = dict(horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        class_mode = None,
        color_mode = 'grayscale',
        target_size=(128,128),
        batch_size = batch_size,
        seed = seed)

    
    train_generator = image_generator
    
    for img in train_generator:
        img2 = preprocess_data(img)
        yield (img2)

In [ ]:
test_img_path = '/content/gdrive/MyDrive/data2/test/non'
test_img_gen = trainGenerator(test_img_path)

In [ ]:
test_image_batch = test_img_gen.__next__()

Found 3000 images belonging to 1 classes.


In [ ]:

images100 = []
j = 0
for i in range(3000):
  j = j + 1
  test_img = test_image_batch[i]

  test_img_norm=test_img[:,:,0][:,:,None]
  test_img_input=np.expand_dims(test_img_norm, 0)
  print(j)
  prediction = (model_unet.predict(test_img_input))
  predicted_img=np.argmax(prediction, axis=3)[0,:,:]
  images100.append(predicted_img)



In [ ]:
print(len(images100))

3000


In [ ]:
import time

i = 0
for item in images100:
  img_w, img_h = 128, 128
  data = np.zeros((img_h, img_w,3), dtype=np.uint8)
  data = item
  img = Image.fromarray((data * 255).astype(np.uint8))
  img.save('/content/gdrive/MyDrive/data3/test/non/non{}.png'.format(i))
  time.sleep(0.2) 
  print(i)
  i = i +1

# **Test covid**

In [ ]:
seed=24
batch_size= 100
n_classes=2

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
#Define a function to perform additional preprocessing after datagen.
#For example, scale images, convert masks to categorical, etc. 
def preprocess_data(img):
    #Scale images
    img = img / 255. 

      
    return img

#Define the generator.
#We are not doing any rotation or zoom to make sure mask values are not interpolated.
#It is important to keep pixel values in mask as 0, 1, 2, 3, .....
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def trainGenerator(train_img_path):
    
    img_data_gen_args = dict(horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        class_mode = None,
        color_mode = 'grayscale',
        target_size=(128,128),
        batch_size = batch_size,
        seed = seed)
    

    train_generator = image_generator
    
    for img in train_generator:
        img2 = preprocess_data(img)
        yield (img2)

In [ ]:
test_img_path = '/content/gdrive/MyDrive/data2/test/covid'
test_img_gen = trainGenerator(test_img_path)

In [ ]:
test_image_batch = test_img_gen.__next__()

Found 100 images belonging to 1 classes.


In [ ]:

images200 = []
j = 0
for i in range(100):
  j = j + 1
  test_img = test_image_batch[i]
  test_img_norm=test_img[:,:,0][:,:,None]
  test_img_input=np.expand_dims(test_img_norm, 0)
  print(j)
  prediction = (model_unet.predict(test_img_input))
  predicted_img=np.argmax(prediction, axis=3)[0,:,:]
  images200.append(predicted_img)



In [ ]:
print(len(images200))

100


In [ ]:
import time

i = 0
for item in images200:
  img_w, img_h = 128, 128
  data = np.zeros((img_h, img_w,3), dtype=np.uint8)
  data = item
  img = Image.fromarray((data * 255).astype(np.uint8))
  img.save('/content/gdrive/MyDrive/data3/test/covid/{}.png'.format(i))
  time.sleep(0.2) 
  print(i)
  i = i +1

# **Check**

In [ ]:
print(len(os.listdir('/content/gdrive/MyDrive/data3/train/covid')))
print(len(os.listdir('/content/gdrive/MyDrive/data3/train/non')))
print(len(os.listdir('/content/gdrive/MyDrive/data3/test/covid')))
print(len(os.listdir('/content/gdrive/MyDrive/data3/test/non')))

420
2000
100
3000
